In [4]:
from gensim.models import Word2Vec
from gensim.test.utils import common_texts

In [12]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [15]:
# wv.save("word2vec-google-news-300.model")


In [ ]:
# http://deeplearning.net/datasets/
# http://www.cs.utoronto.ca/~kriz/cifar.html

In [16]:
import tensorflow as tf
from tensorflow import keras

tf.__version__, keras.__version__

('2.0.0', '2.2.4-tf')

In [17]:
data = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 196s 1us/step


In [61]:
(train_images, train_labels), (test_images, test_labels) = data

class_names = [
    'airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse',
    'ship', 'truck'
]
np.unique(train_labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [67]:
train_images = train_images[(train_labels <= 6).ravel()]
train_labels = train_labels[train_labels <= 6]

In [68]:
test_images = test_images[(test_labels > 6).ravel()]
train_labels = test_labels[test_labels > 6]

((35000, 32, 32, 3), (3000, 32, 32, 3))